# NICT Badge & Wallet System — Code Examples


This notebook provides practical examples that reflect the workflows described in [`docs/workflows.md`](./workflows.md), including:
- User Registration
- Admin Registration
- User Information Update
- NFT Definition Creation
- NFT Issuance to a User
- NFT Synchronization from the Blockchain
- User Bingo Card Info Update
- Bingo Card Information Request
- Prize Draw Type Setup
- Prize Draw Winning Number Submission
- Prize Draw Evaluation
- Workflow: Prize Draw Ranking

Before you start, make sure to set up your Python environment and initialize the database as described in the [Quick Start](../README.md#quick-start) section of README.

---


---
## Working with the Database

Most database operations use **SQLAlchemy ORM 2.0 style** sessions created via a `sessionmaker`. The pattern keeps transactions tidy and consistent.


### Step 1: Setup

Create an engine and sessionmaker:


In [ ]:
from nictbw.db.engine import make_engine
from sqlalchemy.orm import sessionmaker

engine = make_engine()  # This uses the DB_URL environment variable
Session = sessionmaker(engine)

### Step 2: Use the Session

Wrap your code inside a `with` block:


In [ ]:
with Session.begin() as session:
    # Your DB code goes here
    # ...
    pass  # Replace with real work in your application.

This pattern **automatically commits** if everything succeeds or **rolls back** if an exception is raised. 

You can also call `session.flush()` inside the block to write changes to the database before the block closes. This is useful when you need to access auto-generated fields (like primary keys) immediately after an insert.


---
## Notebook Helpers

The examples below generate unique identifiers so you can re-run cells without conflicting with previously inserted rows. Run the helper cell once before exploring the workflows.


In [ ]:
import uuid

EXAMPLE_SUFFIX = uuid.uuid4().hex[:8]
WORKFLOW_USER_IN_APP_ID = f"workflow_user_{EXAMPLE_SUFFIX}"
WORKFLOW_USER_SIGNUP_USERNAME = WORKFLOW_USER_IN_APP_ID
WORKFLOW_USER_SIGNUP_EMAIL = f"{WORKFLOW_USER_SIGNUP_USERNAME}@example.com"
WORKFLOW_USER_SIGNUP_PASSWORD = "ChangeMe123!"
WORKFLOW_USER_PAYMAIL = None
WORKFLOW_ADMIN_EMAIL = f"admin+{EXAMPLE_SUFFIX}@example.com"
WORKFLOW_NFT_PREFIX = f"WF-{EXAMPLE_SUFFIX}"
WORKFLOW_NFT_NAME = f"{EXAMPLE_SUFFIX}-Badge"
WORKFLOW_NFT_SUBCATEGORY = f"{EXAMPLE_SUFFIX}-shop"
WORKFLOW_SHARED_KEY = f"shared-{EXAMPLE_SUFFIX}"
WORKFLOW_PRIZE_DRAW_INTERNAL_NAME = f"workflow_draw_{EXAMPLE_SUFFIX}"
WORKFLOW_PRIZE_DRAW_DISPLAY_NAME = f"Workflow Draw {EXAMPLE_SUFFIX}"
WORKFLOW_PRIZE_DRAW_WINNING_NUMBER = f"WIN-{EXAMPLE_SUFFIX}"


print("Using example suffix:", EXAMPLE_SUFFIX)
print("User in_app_id:", WORKFLOW_USER_IN_APP_ID)
print("Blockchain username:", WORKFLOW_USER_SIGNUP_USERNAME)
print("Blockchain email:", WORKFLOW_USER_SIGNUP_EMAIL)
print("User paymail (populated after registration):", WORKFLOW_USER_PAYMAIL)
print("Admin email:", WORKFLOW_ADMIN_EMAIL)
print("NFT prefix:", WORKFLOW_NFT_PREFIX)
print("Shared key:", WORKFLOW_SHARED_KEY)
print("Prize draw internal name:", WORKFLOW_PRIZE_DRAW_INTERNAL_NAME)
print("Prize draw display name:", WORKFLOW_PRIZE_DRAW_DISPLAY_NAME)
print("Winning number value:", WORKFLOW_PRIZE_DRAW_WINNING_NUMBER)


### Shared Imports

Load the models and workflows used throughout the notebook:


In [ ]:
from nictbw.models import (
    Admin,
    NFTDefinition,
    NFTInstance,
    PrizeDrawResult,
    PrizeDrawType,
    PrizeDrawWinningNumber,
    User,
)
from nictbw import workflows
from sqlalchemy import select


---
## Workflow Examples

Each example below refers to a workflow described in [`docs/workflows.md`](./workflows.md).

Note that if you re-run cells, you might run into unique constraint violations (e.g., duplicate emails or paymails). To solve this, simply re-run the helper cell above to generate a new unique suffix.

---


### Workflow: User Registration


In [ ]:
with Session.begin() as session:
    workflow_user = User(
        in_app_id=WORKFLOW_USER_IN_APP_ID,
        nickname="Workflow Explorer",
    )
    workflows.register_user(
        session,
        workflow_user,
        password=WORKFLOW_USER_SIGNUP_PASSWORD,
        email=WORKFLOW_USER_SIGNUP_EMAIL,
    )

    print(
        f"Registered user id={workflow_user.id} (paymail={workflow_user.paymail}, "
        f"on_chain_id={workflow_user.on_chain_id})"
    )

### Workflow: Admin Registration

In [ ]:
with Session.begin() as session:
    admin = Admin(
        email=WORKFLOW_ADMIN_EMAIL,
        password_hash="hashed-password-placeholder",
        name="Workflow Admin",
        role="event_manager",
    )
    session.add(admin)
    session.flush()  # Ensure admin.id is populated since we need it in the print statement
    print(f"Created admin id={admin.id}")


### Workflow: User Information Update

Load the user and adjust the attributes you need.

In [ ]:
with Session.begin() as session:
    user = User.get_by_in_app_id(session, WORKFLOW_USER_IN_APP_ID)
    if user is None:
        raise RuntimeError("Run the user registration example first.")

    user.nickname = "Workflow Explorer v2"
    user.set_password_hash("new-password-hash")
    session.flush()

    print(
        f"Updated user nickname to {user.nickname} (password hash set? {user.password_hash is not None})"
    )

### Workflow: NFT Definition Creation

---


In [ ]:
with Session.begin() as session:
    admin = Admin.get_by_email(session, WORKFLOW_ADMIN_EMAIL)
    if admin is None:
        raise RuntimeError("Run the admin registration example first.")

    definition = NFTDefinition(
        prefix=WORKFLOW_NFT_PREFIX,
        shared_key=WORKFLOW_SHARED_KEY,
        name=WORKFLOW_NFT_NAME,
        nft_type="default",
        category="event",
        subcategory=WORKFLOW_NFT_SUBCATEGORY,
        created_by_admin_id=admin.id,
        description="Issued during the workflow walkthrough example.",
        triggers_bingo_card=True,
    )
    session.add(definition)
    session.flush()
    print(
        f"Created NFT definition id={definition.id} (triggers_bingo_card={definition.triggers_bingo_card})"
    )


### Workflow: NFT Creation and Issuance to a User

**Note:** The following example issues an NFT in the local database. If you also mint on-chain,
store chain metadata (e.g. `nft_origin`) on the resulting `NFTInstance`.

---


In [ ]:
with Session.begin() as session:
    user = User.get_by_in_app_id(session, WORKFLOW_USER_IN_APP_ID)
    nft_definition = session.scalar(
        select(NFTDefinition).where(NFTDefinition.prefix == WORKFLOW_NFT_PREFIX)
    )
    if user is None or nft_definition is None:
        raise RuntimeError("Ensure the user and NFT definition exist before issuing.")

    issued_instance = workflows.create_and_issue_instance(
        session=session,
        user=user,
        shared_key=None,
        definition_or_template=nft_definition,
    )

    if not isinstance(issued_instance, NFTInstance):
        raise RuntimeError("Expected create_and_issue_instance to return an NFTInstance.")
    issued_instance.nft_origin = f"origin-{EXAMPLE_SUFFIX}"
    session.flush()

    print(
        f"Issued NFT definition id={issued_instance.definition_id} instance id={issued_instance.id} origin={issued_instance.nft_origin}"
    )
    print(f"User now owns {len(user.nft_instances)} NFT instance(s).")


### Workflow: NFT Synchronization from the Blockchain

Call `User.sync_nft_instances_from_chain` to reconcile the local database with the blockchain.
Use this workflow only when NFT instances may have been minted or transferred on-chain without corresponding local records.

**Requirements:**
- The user must have an `on_chain_id` produced during blockchain registration.
- Valid blockchain API credentials must be available via environment variables.

The helper also updates `NFTDefinition.minted_count`, stores the raw metadata in `NFTInstance.other_meta`,
and ensures `unique_instance_id` is consistent with the `<prefix>-<base62(12)>` format.

---


In [ ]:
with Session.begin() as session:
    user = User.get_by_in_app_id(session, WORKFLOW_USER_IN_APP_ID)
    if user is None:
        raise RuntimeError("Run the user registration example first.")

    if user.on_chain_id is None:
        raise RuntimeError("The user must have an on-chain identifier before syncing.")

    before_instances = len(user.nft_instances)

    user.sync_nft_instances_from_chain(session)

    session.flush()
    session.expire(user, ["nft_instances"])

    refreshed_instances = list(user.nft_instances)
    after_instances = len(refreshed_instances)

    print(f"Instance records before sync: {before_instances}")
    print(f"Instance records after sync: {after_instances}")

### Workflow: User Bingo Card Info Update

**Note:** Normally it is not necessary to call these methods directly, as the bingo card and cell info is automatically updated when the user gets a new NFT instance issued via the `create_and_issue_instance` workflow. This workflow is only needed when the bingo card or cell info gets out of sync for some reason.

In [ ]:
with Session.begin() as session:
    user = User.get_by_in_app_id(session, WORKFLOW_USER_IN_APP_ID)
    if user is None:
        raise RuntimeError("Run the user registration example first.")

    before_cards = len(user.bingo_cards)
    workflows.update_user_bingo_info(session, user)
    session.expire(user, ["bingo_cards"])
    after_cards = len(user.bingo_cards)

    print(f"Bingo cards before update: {before_cards}")
    print(f"Bingo cards after update: {after_cards}")

### Workflow: Bingo Card Information Request


You can get the user's bingo card information as a dictionary (JSON) or a JSON string, either in full or in a compact form.

In [ ]:
with Session.begin() as session:
    user = User.get_by_in_app_id(session, WORKFLOW_USER_IN_APP_ID)
    if user is None:
        raise RuntimeError("Run the user registration example first.")

    # full_payload = user.bingo_cards_json()
    # full_payload_str = json.dumps(full_payload, indent=2, ensure_ascii=False)
    compact_payload = user.bingo_cards_json(compact=True)
    compact_payload_str = user.bingo_cards_json_str(compact=True)
    print("Compact JSON string:")
    print(compact_payload_str)

In [ ]:
# For better readability in Jupyter notebooks
compact_payload

### Workflow: Prize Draw Type Setup

Create or reuse a draw type configuration `PrizeDrawType` before storing winning numbers or evaluating NFT instances.

> **Note:**
> A `PrizeDrawType` is essentially a configuration that defines how to evaluate NFT instances for winning. For example, you might have at least two types of prize draws:
> 1. The prize draw that is performed whenever a user gets a new NFT instance. This type typically uses the `"sha256_hex_proximity"` algorithm with a low similarity threshold (close to 0.0) to reward users for collecting NFT instances.
> 2. The prize draw that chooses the user with the closest matching NFT instance as the winner, no matter how similar it is. This is typically performed when a special event occurs and the organizer wants to pick a winner from all NFT holders.
> Coupons and draw scheduling are handled by the API; this backend selects eligible NFT instances and computes winners.

The scoring algorithm is specified by the `PrizeDrawType.algorithm_key` field. The currently supported algorithms (defined in `nictbw.prize_draw.scoring.DEFAULT_SCORING_REGISTRY`) are:
- `"sha256_hex_proximity"`: Hashes both values with SHA-256, interprets the hex digests as large integers, and returns the normalized inverse absolute difference (0.0–1.0, higher is closer).

If necessary, you can also implement and register your own scoring algorithms. See the docstring of `nictbw.prize_draw.scoring.AlgorithmRegistry.register` for details.


In [ ]:
with Session.begin() as session:
    # Retrieve
    draw_type = PrizeDrawType.get_by_internal_name(
        session, WORKFLOW_PRIZE_DRAW_INTERNAL_NAME
    )

    if draw_type is None:
        # Create
        draw_type = PrizeDrawType(
            internal_name=WORKFLOW_PRIZE_DRAW_INTERNAL_NAME,
            display_name=WORKFLOW_PRIZE_DRAW_DISPLAY_NAME,
            description="Demo draw type created from the workflow examples.",
            algorithm_key="sha256_hex_proximity",  # specify the score calculating algorithm to use
            default_threshold=0.8,
        )

        session.add(draw_type)
        session.flush()
        print(f"Created draw type id={draw_type.id}")

    else:
        print(f"Reusing existing draw type id={draw_type.id}")

    WORKFLOW_PRIZE_DRAW_TYPE_ID = draw_type.id

### Workflow: Adding Prize Draw Winning Number

Submit a winning number for the draw type created above. The helper `nictbw.workflows.submit_winning_number` persists the value and returns the ORM entity.

In [ ]:
with Session.begin() as session:
    # Prepare the draw type
    draw_type = PrizeDrawType.get_by_internal_name(
        session, WORKFLOW_PRIZE_DRAW_INTERNAL_NAME
    )
    if draw_type is None:
        raise RuntimeError("Run the prize draw type setup example first.")

    # Submit a winning number
    winning_number = workflows.submit_winning_number(
        session,
        draw_type,
        value=WORKFLOW_PRIZE_DRAW_WINNING_NUMBER,
    )

    # And it's done
    # The below is only for later reference in the notebook
    WORKFLOW_PRIZE_DRAW_WINNING_NUMBER_ID = winning_number.id

    print(
        "Stored winning number id={id} for draw type '{name}' with value='{value}'".format(
            id=winning_number.id,
            name=draw_type.internal_name,
            value=winning_number.value,
        )
    )

### Workflow: Single Prize Draw Evaluation and Bulk Evaluation

`workflows.run_prize_draw` evaluates an `NFTInstance` against the configured draw type and winning number, saving the result as a `PrizeDrawResult` entity into the database. Re-running the helper will overwrite the prior result for the same combination.

For batches, use `workflows.run_prize_draw_batch` (defaults to instances on completed bingo lines when `nft_instances` is omitted), or the derived helpers `run_bingo_prize_draw` and `run_final_attendance_prize_draw` described below.


In [ ]:
with Session.begin() as session:
    # Prepare the draw type, winning number, and user.
    # (This is only an example,
    # you might retrieve these using different methods in your application.)
    draw_type = session.get(PrizeDrawType, WORKFLOW_PRIZE_DRAW_TYPE_ID)
    winning_number = session.get(
        PrizeDrawWinningNumber, WORKFLOW_PRIZE_DRAW_WINNING_NUMBER_ID
    )
    user = User.get_by_in_app_id(session, WORKFLOW_USER_IN_APP_ID)
    if draw_type is None or winning_number is None or user is None:
        raise RuntimeError(
            "Ensure the draw type, winning number, and user exist before evaluating."
        )
    instance = next(
        (o for o in user.nft_instances if o.definition is not None and o.definition.shared_key == WORKFLOW_SHARED_KEY),
        None,
    )
    if instance is None:
        raise RuntimeError(
            "Run the NFT creation and issuance example before evaluating the draw."
        )

    # Evaluate the draw
    result = workflows.run_prize_draw(
        session,
        nft_instance=instance,
        draw_type=draw_type,
        winning_number=winning_number,
        threshold=draw_type.default_threshold,
    )

    # The below is only for later reference in the notebook
    WORKFLOW_PRIZE_DRAW_RESULT_ID = result.id
    print(
        "Result id={id} outcome={outcome} score={score} threshold={threshold}".format(
            id=result.id,
            outcome=result.outcome,
            score=result.similarity_score,
            threshold=result.threshold_used,
        )
    )
    print(f"Draw number: {result.draw_number}")
    print("Top digits: draw={draw} winning={win}".format(
        draw=result.draw_top_digits or "0000000000",
        win=result.winning_top_digits or "0000000000",
    ))


To evaluate multiple NFT instances in one call, you can use `workflows.run_prize_draw_batch` instead.

To do that, simply replace the `run_prize_draw` above with `run_prize_draw_batch`, passing a list of instances instead of a single instance. If the `nft_instances` argument is omitted, it evaluates all eligible instances that belong to completed bingo-card lines. For scheduled draws, prefer `run_bingo_prize_draw` (completed lines) or `run_final_attendance_prize_draw` (final-day attendance definition).

---


In [ ]:
with Session.begin() as session:
    draw_type = session.get(PrizeDrawType, WORKFLOW_PRIZE_DRAW_TYPE_ID)

    workflows.run_prize_draw_batch(session, draw_type=draw_type)

### Workflow: Bingo Prize Draw (Completed Lines)

Use `workflows.run_bingo_prize_draw` when a draw should consider **only NFT instances that sit on completed bingo lines**. Results are ranked by similarity, and ties at the cutoff are all included when `limit` is set.

In [ ]:
with Session.begin() as session:
    draw_type = session.get(PrizeDrawType, WORKFLOW_PRIZE_DRAW_TYPE_ID)
    winning_number = session.get(PrizeDrawWinningNumber, WORKFLOW_PRIZE_DRAW_WINNING_NUMBER_ID)

    winners = workflows.run_bingo_prize_draw(
        session,
        draw_type,
        winning_number=winning_number,
        limit=5,
    )

    print("Bingo draw winner definition ids:", [w.definition_id for w in winners])


### Workflow: Final-Day Attendance Prize Draw

Use `workflows.run_final_attendance_prize_draw` when the draw should target only the final-day attendance stamp NFT instances. Provide the final-day attendance NFT definition id, and the helper will evaluate only NFT instances minted from that definition, returning ties at the cutoff when `limit` is set.

---


In [ ]:
attendance_definition_id = 123  # replace with your final-day attendance NFT definition id

with Session.begin() as session:
    draw_type = session.get(PrizeDrawType, WORKFLOW_PRIZE_DRAW_TYPE_ID)
    winning_number = session.get(PrizeDrawWinningNumber, WORKFLOW_PRIZE_DRAW_WINNING_NUMBER_ID)

    winners = workflows.run_final_attendance_prize_draw(
        session,
        draw_type,
        attendance_definition_id=attendance_definition_id,
        winning_number=winning_number,
        limit=5,
    )

    print("Final-day attendance winner definition ids:", [w.definition_id for w in winners])



### Workflow: Prize Draw Ranking

When a draw type does not rely on thresholds (for example, when finding a "closest-number win" ), you can rank the evaluated results and pick the top records.

In [ ]:
with Session.begin() as session:
    # Prepare the draw type
    draw_type = session.get(PrizeDrawType, WORKFLOW_PRIZE_DRAW_TYPE_ID)
    if draw_type is None:
        raise RuntimeError("Run the prize draw type setup example first.")

    # Retrieve top results
    res = workflows.select_top_prize_draw_results(
        session=session, draw_type=draw_type, limit=10
    )

    for r in res:
        print(r)